<a href="https://colab.research.google.com/github/AleAguirreM/Analitica3AA/blob/main/c_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib

In [ ]:
conn = sql.connect("/content/drive/MyDrive/db_movies")

In [ ]:
curr = conn.cursor()

In [ ]:
curr.execute("SELECT name FROM sqlite_master WHERE type = 'table'")

In [ ]:
curr.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('users_final',),
 ('full_ratings',)]

In [ ]:
pd.read_sql("select * from ratings", conn)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
#try:
    # Intentar agregar la columna fecha_fecha si no existe
    #curr.execute("ALTER TABLE ratings ADD COLUMN fecha_fecha DATE")
#except sql.OperationalError as e:
    # Si la columna ya existe, mostrar un mensaje de advertencia
    #print("La columna 'fecha_fecha' ya existe en la tabla 'ratings'.")

# Actualizar la nueva columna con los valores convertidos de la columna existente
#curr.execute("UPDATE ratings SET fecha_fecha = DATE(timestamp)")


La columna 'fecha_fecha' ya existe en la tabla 'ratings'.


In [ ]:
pd.read_sql("select * from movies", conn)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
######################################################################
################## 1. sistemas basados en popularidad ###############
#####################################################################


##### recomendaciones basado en popularidad ######

#### mejores calificadas que tengan calificacion
pd.read_sql("""select title,
            avg(rating) as avg_rat,
            count(*) as read_num
            from full_ratings
            where rating<>0
            group by title
            order by avg_rat desc
            limit 10
            """, conn)

DatabaseError: Execution failed on sql 'select title,
            avg(rating) as avg_rat,
            count(*) as read_num
            from ratings
            where rating<>0
            group by title
            order by avg_rat desc
            limit 10
            ': no such column: title

In [ ]:
#### los mas leidos con promedio de los que calficaron ###
pd.read_sql("""select title,
            avg(iif(book_rating = 0, Null, book_rating)) as avg_rat,
            count(*) as read_num
            from full_ratings
            group by title
            order by read_num desc
            """, conn)

In [ ]:

#######################################################################
######## 2.1 Sistema de recomendación basado en contenido un solo producto - Manual ########
#######################################################################

movies=pd.read_sql('select * from movies_final', conn )

movies.info()
movies['year_pub']=movies.year_pub.astype('int')
movies.info()

##### escalar para que año esté en el mismo rango ###

sc=MinMaxScaler()
movies[["year_sc"]]=sc.fit_transform(movies[['year_pub']])



In [ ]:
## eliminar filas que no se van a utilizar ###

movies_dum1=movies.drop(columns=['userId', 'movieId', 'ratings'])


In [ ]:
#### convertir a dummies

movies_dum1['gender'].nunique()
movies_dum1['publisher'].nunique()

col_dum=['book_author','publisher']
books_dum2=pd.get_dummies(books_dum1,columns=col_dum)
books_dum2.shape

joblib.dump(books_dum2,"salidas\\books_dum2.joblib") ### para utilizar en segundos modelos
